# Check whether a match's charity headquarters (if available) match a shell company's address

In [77]:
#Imports
import pandas as pd

#spark
import findspark
findspark.init(r"C:\Users\Lucas\Desktop\ADA\spark-2.3.2-bin-hadoop2.7")

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min
from pyspark.sql.functions import udf
from pyspark.sql.functions import split
from pyspark.sql.functions import explode
from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField



from pyspark.sql.types import StringType
from pyspark.sql.types import TimestampType

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()

import networkx as nx

from operator import itemgetter
import matplotlib.pyplot as plt
import collections
import community
from networkx.algorithms.community.centrality import girvan_newman
import itertools
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

AttributeError: 'str' object has no attribute '_jsc'

In [68]:
def get_match_file(leak, charity):
    return '../generated/matches/' + leak + '_' + charity + '_matches.csv'

#Load matches
paradise_wiki_matches = spark.read.csv(get_match_file('paradise', 'wikipedia'), header=True)
paradise_ingo_matches = spark.read.csv(get_match_file('paradise', 'INGO'), header=True)
paradise_forbes_matches = spark.read.csv(get_match_file('paradise', 'forbes'), header=True)

matches = paradise_wiki_matches.union(paradise_forbes_matches).union(paradise_ingo_matches).drop('_c0')

#Load edges
edges = spark.read.csv('../data/paradise/paradise_papers.edges.csv', header=True)

address_nodes = spark.read.csv('../data/paradise/paradise_papers.nodes.address.csv', header=True)

In [63]:
matches.take(10)

[Row(node_id='100301845', ShellName='SERVICES INTERNATIONAL LIMITED', CharityName='Population Services International', CharityHeadquarters='Washington, District of Columbia'),
 Row(node_id='100310580', ShellName='J. J. J. ENTERPRISES INC', CharityName='J. Paul Getty Trust', CharityHeadquarters=None),
 Row(node_id='100318846', ShellName='VISION WORLD INC.', CharityName='World Vision International', CharityHeadquarters=None),
 Row(node_id='100318846', ShellName='VISION WORLD INC.', CharityName='World Vision', CharityHeadquarters='Federal Way, Washington'),
 Row(node_id='100318846', ShellName='VISION WORLD INC.', CharityName='World Vision International', CharityHeadquarters=None),
 Row(node_id='100318846', ShellName='VISION WORLD INC.', CharityName='World Vision International', CharityHeadquarters=None),
 Row(node_id='100338311', ShellName='THE LEARNING FOUNDATION INC.', CharityName='The Canadian International Learning Foundation', CharityHeadquarters=None),
 Row(node_id='101208524', Shel

In [54]:
matches.count()

106

In [58]:
address_edges.take(1)

[Row(START_ID='85004927', TYPE='registered_address', END_ID='88000379', link='registered address', start_date=None, end_date=None, sourceID='Paradise Papers - Aruba corporate registry', valid_until='Aruba corporate registry data is current through 2016')]

In [10]:
print('All edges: ', edges.count())
address_edges = edges.rdd.filter(lambda r: r[1] == 'registered_address')
print('Address edges: ', address_edges.count())

All edges:  1657839
Address edges:  450551


In [11]:
def filter_edges(edges, nodes):
    '''Given a set of nodes, returns the edges connected to those nodes'''
    entity_ids = nodes.map(lambda r: r[0]).collect()
    return edges.filter(lambda r: r[0] in entity_ids or r[2] in entity_ids)

entity_address_edges = filter_edges(address_edges, matches.rdd)


In [60]:
entity_address_edges.count()

70

In [13]:
def filter_nodes(nodes, edges):
    '''Given a set of edges, return the nodes connected to those edges'''
    start_ids = edges.map(lambda r: r[0]).collect()
    end_ids = edges.map(lambda r: r[2]).collect()
    
    return nodes.rdd.filter(lambda r: r[0] in start_ids or r[0] in end_ids)

In [14]:
paradise_address=filter_nodes(address_nodes,entity_address_edges)

In [61]:
paradise_address.count()

63

In [15]:
paradise_address.take(1)

[Row(node_id='81027090', name="Canon's Court; 22 Victoria Street; Hamilton; HM 12; Bermuda", address="Canon's Court", country_codes='BMU', countries='Bermuda', sourceID='Paradise Papers - Appleby', valid_until='Appleby data is current through 2014', note=None)]

In [56]:
paradise_address

PythonRDD[196] at RDD at PythonRDD.scala:52

In [78]:
schema = StructType([StructField(str(i), StringType(), True) for i in range(8)])

df = spark.createDataFrame(paradise_address)



In [17]:
df=df.orderBy('node_id',ascending=True)
matches=matches.orderBy('node_id',ascending=True)

df.take(5)

[Row(node_id='120000395', name='EPWORTH CHAMBERS, PINFOLD STREET, BRIDGETOWN , BARBADOS,', address='EPWORTH CHAMBERS, PINFOLD STREET, BRIDGETOWN , BARBADOS,', country_codes='BRB', countries='Barbados', sourceID='Paradise Papers - Barbados corporate registry', valid_until='Barbados corporate registry data is current through 2016', note=None),
 Row(node_id='120001585', name='CHELTENHAM LODGE FONTABELLE ST. MICHAEL BARBADOS', address='CHELTENHAM LODGE FONTABELLE ST. MICHAEL BARBADOS', country_codes='BRB', countries='Barbados', sourceID='Paradise Papers - Barbados corporate registry', valid_until='Barbados corporate registry data is current through 2016', note=None),
 Row(node_id='120003377', name='GOODLAND MAIN ROAD, ST.MICHAEL, BARBADOS.', address='GOODLAND MAIN ROAD, ST.MICHAEL, BARBADOS.', country_codes='BRB', countries='Barbados', sourceID='Paradise Papers - Barbados corporate registry', valid_until='Barbados corporate registry data is current through 2016', note=None),
 Row(node_id='

In [18]:
matches_pd=pd.DataFrame(matches.collect())
df_pd=pd.DataFrame(df.collect())

In [19]:
matches.show()

+---------+--------------------+--------------------+--------------------+
|  node_id|           ShellName|         CharityName| CharityHeadquarters|
+---------+--------------------+--------------------+--------------------+
|100301845|SERVICES INTERNAT...|Population Servic...|Washington, Distr...|
|100310580|J. J. J. ENTERPRI...| J. Paul Getty Trust|                null|
|100318846|   VISION WORLD INC.|        World Vision|Federal Way, Wash...|
|100318846|   VISION WORLD INC.|World Vision Inte...|                null|
|100318846|   VISION WORLD INC.|World Vision Inte...|                null|
|100318846|   VISION WORLD INC.|World Vision Inte...|                null|
|100338311|THE LEARNING FOUN...|The Canadian Inte...|                null|
|101208524|THE NATURE CONSER...|  Nature Conservancy| Arlington, Virginia|
|101422196|INTERNATIONAL RES...|International Res...|  New York, New York|
|101510864|EPITAPH'S INVISIB...|Invisible Childre...|                null|
|101803231|ROYAL SOCIETY 

In [20]:
df.show()

+---------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------+
|  node_id|                name|             address| country_codes|           countries|            sourceID|         valid_until|          note|
+---------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------+
|120000395|EPWORTH CHAMBERS,...|EPWORTH CHAMBERS,...|           BRB|            Barbados|Paradise Papers -...|Barbados corporat...|          null|
|120001585|CHELTENHAM LODGE ...|CHELTENHAM LODGE ...|           BRB|            Barbados|Paradise Papers -...|Barbados corporat...|          null|
|120003377|GOODLAND MAIN ROA...|GOODLAND MAIN ROA...|           BRB|            Barbados|Paradise Papers -...|Barbados corporat...|          null|
|120003485|     """WEST RIDGE""|                KENT| CHRIST CHURCH|          BARBADOS."|     """WEST RIDGE""|        

In [101]:
df_pd=df.toPandas()
matches_pd=matches.toPandas()

In [35]:
list_1=df_pd['node_id']
list_2=matches_pd['node_id']

In [40]:
list_1=list_1.astype(int)
list_2=list_2.astype(int)

In [64]:
list_1


0     120000395
1     120001585
2     120003377
3     120003485
4     120004193
5     120004633
6     120006158
7     120008591
8     120013595
9     120016319
10    120016424
11    120017700
12    120018981
13    220000065
14    220000090
15     33000187
16     33000219
17     58041242
18     58045699
19     58052025
20     58060358
21     58084388
22     58089732
23     58129245
24     81000811
25     81010195
26     81010529
27     81010536
28     81010818
29     81027090
        ...    
33     81084646
34     88002083
35     88002434
36     88004102
37     88004413
38     88006236
39     88006821
40     88006824
41     88007075
42     88007171
43     88007521
44     88009040
45     88009300
46     88009368
47     88010094
48     88010106
49     88011038
50     88011063
51     88011305
52     88011332
53     88011451
54     88012062
55     88013209
56     88013778
57     88015505
58     88016029
59     88016305
60     88017290
61     88017324
62     88017348
Name: node_id, Length: 6

In [48]:
ds1 = pd.Series([2, 4, 3, 8, 10])
ds2 = pd.Series([1, 3, 5, 7, 10])

In [66]:
ds1.isin(ds2)

0    False
1    False
2     True
3    False
4     True
dtype: bool

In [67]:
list_1[list_1.isin(list_2)]

Series([], Name: node_id, dtype: int32)

In [135]:
address_pd=df_pd

In [99]:
entity_address_edges_pd=pd.DataFrame(entity_address_edges.collect(), columns=["node_id","TYPE","END_ID","link","start_date","end_date","sourceID","valid_until"])

In [108]:
matches_pd['node_id']=matches_pd['node_id'].apply(int)
entity_address_edges_pd['node_id']=entity_address_edges_pd['node_id'].apply(int)

In [124]:
display(matches_pd.head(1))
display(entity_address_edges_pd.head(1))
display(address_pd.head(1))

,node_id,ShellName,CharityName,CharityHeadquarters
0,82010944,Managed Care,CARE,None


,node_id,TYPE,END_ID,link,start_date,end_date,sourceID,valid_until
0,82001786,registered_address,81027090,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014


,node_id,name,address,country_codes,countries,sourceID,valid_until,note
0,120000395,"EPWORTH CHAMBERS, PINFOLD STREET, BRIDGETOWN ,...","EPWORTH CHAMBERS, PINFOLD STREET, BRIDGETOWN ,...",BRB,Barbados,Paradise Papers - Barbados corporate registry,Barbados corporate registry data is current th...,None


In [118]:
prejudice=matches_pd.merge(entity_address_edges_pd, on ='node_id')

In [123]:
prejudice.rename(columns={'node_id':'START_ID'},inplace=True)

In [125]:
prejudice.head(1)

,START_ID,ShellName,CharityName,CharityHeadquarters,TYPE,END_ID,link,start_date,end_date,sourceID,valid_until
0,82010944,Managed Care,CARE,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014


In [138]:
#case of start ID
address_pd=df_pd
address_pd.drop(['note','valid_until','sourceID'],axis=1,inplace=True)
address_pd.rename(columns={'node_id':'START_ID'},inplace=True)
address_pd['START_ID']=address_pd['START_ID'].apply(int)

In [139]:
pride_start=prejudice.merge(address_pd, on ='START_ID')

In [141]:
pride_start

,START_ID,ShellName,CharityName,CharityHeadquarters,TYPE,END_ID,link,start_date,end_date,sourceID,valid_until,name,address,country_codes,countries


In [143]:
#case of end ID
address_pd.rename(columns={'START_ID':'END_ID'},inplace=True)
prejudice['END_ID']=prejudice['END_ID'].apply(int)
pride_end=prejudice.merge(address_pd, on ='END_ID')

In [146]:
prejudice.head(2)

,START_ID,ShellName,CharityName,CharityHeadquarters,TYPE,END_ID,link,start_date,end_date,sourceID,valid_until
0,82010944,Managed Care,CARE,None,registered_address,81027146,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014
1,82010944,Managed Care,CARE,None,registered_address,81010536,registered office,None,None,Paradise Papers - Appleby,Appleby data is current through 2014


In [145]:
address_pd

,END_ID,name,address,country_codes,countries
0,81027090,Canon's Court; 22 Victoria Street; Hamilton; H...,Canon's Court,BMU,Bermuda
1,81027146,Clifton House; 75 Fort Street; Grand Cayman KY...,Clifton House,CYM,Cayman Islands
2,81029389,Argyle House; 41a Cedar Avenue; Hamilton HM 12...,Argyle House,BMU,Bermuda
3,81031545,46 Point Finger Road; Paget DV 04; Bermuda,46 Point Finger Road,BMU,Bermuda
4,81084646,c/o Sonnedix UK Services Limited; 57 Great Suf...,c/o Sonnedix UK Services Limited,GBR,United Kingdom
5,81000811,Heiligkreuz 6; Vaduz; 9490; Liechtenstein,Heiligkreuz 6,LIE,Liechtenstein
6,81010195,"PO Box 852 ; UBS House, 227 Elgin Avenue; Geor...",PO Box 852,CYM,Cayman Islands
7,81010529,PO Box 1348; KY1-1108; Grand Cayman; Cayman Is...,PO Box 1348,CYM,Cayman Islands
8,81010536,Kirk House 3rd Floor; Albert Panton Street; Ge...,Kirk House 3rd Floor,CYM,Cayman Islands
9,81010818,Rua das Faias 978; Quinta da Bicuda; 2750-688 ...,Rua das Faias 978,PRT,Portugal
